Importing section

In [98]:
import os
import string
from nltk import tokenize
from gensim.models import Word2Vec
from nltk import word_tokenize
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples
import pandas as pd

Reading the data from the folder and cleaning it.

In [99]:
sentences = []
for filename in os.listdir(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\TelevisonMergedT1+T2"):
   with open(os.path.join(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\TelevisonMergedT1+T2", filename)) as f:
       text = f.read()
       text = text.replace("ï»¿","")
       sents = tokenize.sent_tokenize(text)
       for s in sents:
           s = s.lower()
           s = s.translate(str.maketrans('', '', string.punctuation))
           sentences.append(s)

tokensSentenceslist = []

for s  in sentences:
    wordsList = word_tokenize(s)
    tokensSentenceslist.append(wordsList)


##################### Uncomment below section for testing #########################
# print(len(sentences))
#
# for s in sentences:
#      print("The sentence is : ")
#      print(s)
#      print("-----------------------End of the sentence -------------")
#
# print (sentences)


# print (len(tokensSentenceslist))
# print (tokensSentenceslist)


In [100]:
print (len(sentences))
print(sentences[0:5])

38715
['advertising in television is becoming more and more clever', 'lots of those ads invade our subconsciousness and try to make us buy the product even if we do not actually need it', 'when it comes to children lots of them dont have enough money to afford these products', 'they dont even understand the concept of money', 'even if they wanted it they can not get the thing they were told they need']


Generating the Word2Vec Model

In [101]:
#model = Word2Vec(tokensSentenceslist, min_count=1)

#model = Word2Vec(tokensSentenceslist, vector_size=50, min_count=1, sg=1)
#model = Word2Vec(sentences=tokensSentenceslist, vector_size=100, workers=1, seed=42)

model = Word2Vec(window=10, min_count=2,workers=6,vector_size=50,seed=42,sg=0)
model.build_vocab(tokensSentenceslist, progress_per=1000)
model.train(tokensSentenceslist, total_examples=model.corpus_count, epochs=model.epochs)


##################### Uncomment below section for testing #########################


# print(list(model.wv.index_to_key))
# print(len(list(model.wv.index_to_key)))

(2373525, 3636460)

In [63]:
model.corpus_count
#model.epochs

38715

In [ ]:
model.wv.most_similar("television")
model.wv.most_similar("argument")
model.wv.similarity("tv","against")

In [105]:
def vectorize(list_of_docs, model, strategy):
    """Generate vectors for list of documents using a Word Emx`bedding.

    Args:
        list_of_docs: List of documents.
        model: Gensim Word Embedding.
        strategy: Aggregation strategy ("average", or "min-max".)

    Raises:
        ValueError: If the strategy is other than "average" or "min-max".

    Returns:
        List of vectors.
    """
    features = []
    size_output = model.vector_size
    embedding_dict = model

    if strategy == "min-max":
        size_output *= 2

    if hasattr(model, "wv"):
        embedding_dict = model.wv

    for tokens in list_of_docs:
        zero_vector = np.zeros(size_output)
        vectors = []
        for token in tokens:
            if token in embedding_dict:
                try:
                    vectors.append(embedding_dict[token])
                except KeyError:
                    continue
        if vectors:
            vectors = np.asarray(vectors)
            if strategy == "min-max":
                min_vec = vectors.min(axis=0)
                max_vec = vectors.max(axis=0)
                features.append(np.concatenate((min_vec, max_vec)))
            elif strategy == "average":
                avg_vec = vectors.mean(axis=0)
                features.append(avg_vec)
            else:
                raise ValueError(f"Aggregation strategy {strategy} does not exist!")
        else:
            features.append(zero_vector)
    return features

-0.1692446

In [106]:
vectorized_docs = vectorize(tokensSentenceslist, model=model, strategy="average")
 #len(vectorized_docs), len(vectorized_docs[0])
# print(model.wv["argument"])
# print("#######################################################")
# print(vectorized_docs[0])

In [89]:
def mbkmeans_clusters(X, k, mb=500, print_silhouette_values=False):
    """Generate clusters.

    Args:
        X: Matrix of features.
        k: Number of clusters.
        mb: Size of mini-batches. Defaults to 500.
        print_silhouette_values: Print silhouette values per cluster.

    Returns:
        Trained clustering model and labels based on X.
    """
    km = MiniBatchKMeans(n_clusters=k, batch_size=mb).fit(X)
    print(f"For n_clusters = {k}")
    print(f"Silhouette coefficient: {silhouette_score(X, km.labels_):0.2f}")
    print(f"Inertia:{km.inertia_}")

    if print_silhouette_values:
        sample_silhouette_values = silhouette_samples(X, km.labels_)
        print(f"Silhouette values:")
        silhouette_values = []
        for i in range(k):
            cluster_silhouette_values = sample_silhouette_values[km.labels_ == i]
            silhouette_values.append(
                (
                    i,
                    cluster_silhouette_values.shape[0],
                    cluster_silhouette_values.mean(),
                    cluster_silhouette_values.min(),
                    cluster_silhouette_values.max(),
                )
            )
        silhouette_values = sorted(
            silhouette_values, key=lambda tup: tup[2], reverse=True
        )
        for s in silhouette_values:
            print(
                f"    Cluster {s[0]}: Size:{s[1]} | Avg:{s[2]:.2f} | Min:{s[3]:.2f} | Max: {s[4]:.2f}"
            )
    return km, km.labels_


[ 1.0834987   1.0775832  -2.4190793   1.3539352   0.7693121  -0.01766455
 -0.53748876  2.1623745  -0.5207145   1.694928   -0.2794287   1.4977117
  0.3757558  -1.0392004  -1.1415775  -0.37531468  0.23239893  1.364053
  1.6795251   0.6714833   0.8824193   1.365521    1.8032991  -0.67293257
  1.0104337  -0.03604451  1.0017824  -0.5705173  -0.49769476  1.6430551
 -1.352539    1.2772088   0.03308525 -2.0387952  -2.4316406  -1.2589905
  0.95908076 -2.2138608   0.6789476  -1.0684443   0.03440151  0.9796872
  0.9881222  -1.3170159  -2.6712847  -1.2676587   1.5911027  -0.6523292
 -1.2096913   0.46780825]
#######################################################
[ 0.09301123  0.02892222  0.02476533  0.29560983 -0.7727718   0.23872662
  0.4515218  -0.06879666 -0.03741706 -0.3008877  -0.20542604  0.8528051
 -0.14720449 -0.2436079  -1.0440915  -0.17934558 -0.21951494 -0.6070896
  0.41599515  0.6486483  -0.45739534  0.38212645  0.3662845   0.4625144
  0.98150975 -0.35466102  0.38813674 -0.44521755 -1.

In [108]:
clustering, cluster_labels = mbkmeans_clusters(X=vectorized_docs, k=10, print_silhouette_values=True)

df_clusters = pd.DataFrame({
    "text": sentences,
    "tokens": [" ".join(text) for text in tokensSentenceslist],
    "cluster": cluster_labels
})

### Evaluate top terms of the cluster

In [145]:
print("Top terms per cluster (based on centroids):")
for i in range(10): # number of cluster k should be put here!!
    tokens_per_cluster = ""
    most_representative = model.wv.most_similar(positive=[clustering.cluster_centers_[i]], topn=3)
    print(clustering.cluster_centers_[i][0])
    for t in most_representative:
        tokens_per_cluster += f"{t[0]} "
    #print(f"Cluster {i}: {tokens_per_cluster}")

Top terms per cluster (based on centroids):
[ 1.19385659 -0.82888172  1.58800147  0.10622866 -0.04769936  0.53634079
  1.2926703  -0.58823001  0.37684249 -0.0375403   1.1391392  -0.21388091
 -0.8419543  -0.18153256  0.01115879  0.0577653  -0.66395227  0.1070669
  0.3855902   0.07921372  0.25195381 -0.24987248  0.91514523  0.87854672
 -0.2781197  -1.14427267  0.05927777  0.0140876  -1.56937785  0.24000135
  0.75462581 -0.68548434 -0.51476748  0.84428861 -0.97498851  0.26553429
  0.41518204 -0.35250388 -1.04682077  0.19067908 -0.42692063  0.4745817
  0.7416655  -0.27141488  0.15662372 -0.24953608  0.13406592  0.3687836
 -0.95627766  0.222207  ]
[ 0.70034438 -0.29410066  0.99018799  0.20499822 -0.04243745  0.39972958
  0.64275578 -0.33103261 -0.0416056  -0.13465635  0.72499667  0.00946553
 -0.46231743  0.12923565 -0.10479293 -0.02107713 -0.41081607 -0.20373062
  0.31501559 -0.29613619  0.14241664 -0.22131774  0.29306809  0.58699511
  0.02967619 -0.48050582  0.10401114  0.04557529 -1.17843

### Retrieve a random sample of documents for a given cluster


In [154]:
for i,t in enumerate(df_clusters.query(f"cluster == {0}").sample(10).iterrows()):
    print(t[1]["text"])
    print("-------------")

the parents have the ability and not only that they must protect their children from these influenses
-------------
businsesses need the television advertising to improve the children to buy the toys
-------------
they cant play with their kids
-------------
we might not want to buy the product but we do remember the brand
-------------
and also when children have their toys they dont annoy the parents if theyre bored
-------------
it is simmilar to money if you guard your child from all the money they can not handle it when they are adult but when you give them pocket money and learn them how to handle it they will get use to it
-------------
children think that they are like them
-------------
for children thats a reality they maybe dont have but they want it to be like that
-------------
but this argument can also turned into an argument thats against this statement because the children could learn that they cant always have what they want if the parents persist and dont buy them th

In [151]:
df_clusters.shape

# df_clusters.tokens[0]
# df_clusters.head()

(38715, 3)

### Most representative clusters

In [156]:
test_cluster = 0
most_representative_docs = np.argsort(
    np.linalg.norm(vectorized_docs - clustering.cluster_centers_[test_cluster], axis=1)
)
for d in most_representative_docs[:10]:
    print(sentences[d])
    print("-------------")

if the parents know things better and its up to hope they do they will be the bad guys because they want whitout any measuring reason their kids to not get these toys
-------------
while it can help some people figure out what their children would like to have it usually just leaves the child wanting something that they usually dont get because they cant have every toy they see on tv or because the parents are just not able to afford it
-------------
parents need to tell their children that they should not be greedy and have joy with the things they havethe child will probably understand it and just move on
-------------
the second problem could be that poor parents cant buy their children the toys which they see in the television and then the parents can begin to feel bad and the children thinks their parents dont like them because in this age they often dont understand what poorness is
-------------
parents should also try to teach their children that they cant get anything they see 